# Load Dataset

In [ ]:
#@title Data retrieval
import os, requests

fname = []
for j in range(3):
  fname.append('steinmetz_part%d.npz'%j)
url = ["https://osf.io/agvxh/download"]
url.append("https://osf.io/uv3mw/download")
url.append("https://osf.io/ehmw2/download")

for j in range(len(url)):
  if not os.path.isfile(fname[j]):
    try:
      r = requests.get(url[j])
    except requests.ConnectionError:
      print("!!! Failed to download data !!!")
    else:
      if r.status_code != requests.codes.ok:
        print("!!! Failed to download data !!!")
      else:
        with open(fname[j], "wb") as fid:
          fid.write(r.content)


In [ ]:
#@title Data loading
import numpy as np

alldat = np.array([])
for j in range(len(fname)):
  alldat = np.hstack((alldat, np.load('steinmetz_part%d.npz'%j, allow_pickle=True)['dat']))

# select just one of the recordings here. 11 is nice because it has some neurons in vis ctx.
num_section = 11

dat = alldat[num_section]
print(dat.keys())

dict_keys(['spks', 'wheel', 'pupil', 'response', 'response_time', 'bin_size', 'stim_onset', 'contrast_right', 'contrast_left', 'brain_area', 'feedback_time', 'feedback_type', 'gocue', 'mouse_name', 'date_exp', 'trough_to_peak', 'active_trials', 'contrast_left_passive', 'contrast_right_passive', 'spks_passive', 'pupil_passive', 'wheel_passive', 'prev_reward', 'ccf', 'ccf_axes', 'cellid_orig', 'reaction_time', 'face', 'face_passive', 'licks', 'licks_passive'])


In [ ]:
print(len(alldat))

39


## Combine all sections of data

In [ ]:
for i in range(len(alldat)):
    if i<5:
    # if i == 0:
    #     spks_all = alldat[i]['spks']
    # else:
    #     spks_new = alldat[i]['spks']
    #     spks_all = np.vstack((spks_all, spks_new))
        print('spks',alldat[i]['spks'].shape)
        # print('response',alldat[i]['response'].shape)
        print('---------------------------')

spks (734, 214, 250)
---------------------------
spks (1070, 251, 250)
---------------------------
spks (619, 228, 250)
---------------------------
spks (1769, 249, 250)
---------------------------
spks (1077, 254, 250)
---------------------------


In [ ]:
print(spks_all.shape)

(734, 214, 250)


`alldat` contains 39 sessions from 10 mice, data from Steinmetz et al, 2019. Time bins for all measurements are 10ms, starting 500ms before stimulus onset. The mouse had to determine which side has the highest contrast. For each `dat = alldat[k]`, you have the following fields:

* `dat['mouse_name']`: mouse name
* `dat['date_exp']`: when a session was performed
* `dat['spks']`: neurons by trials by time bins.    
* `dat['brain_area']`: brain area for each neuron recorded. 
* `dat['contrast_right']`: contrast level for the right stimulus, which is always contralateral to the recorded brain areas.
* `dat['contrast_left']`: contrast level for left stimulus. 
* `dat['gocue']`: when the go cue sound was played. 
* `dat['response_time']`: when the response was registered, which has to be after the go cue. The mouse can turn the wheel before the go cue (and nearly always does!), but the stimulus on the screen won't move before the go cue.  
* `dat['response']`: which side the response was (`-1`, `0`, `1`). When the right-side stimulus had higher contrast, the correct choice was `-1`. `0` is a no go response. 
* `dat['feedback_time']`: when feedback was provided. 
* `dat['feedback_type']`: if the feedback was positive (`+1`, reward) or negative (`-1`, white noise burst).  
* `dat['wheel']`: exact position of the wheel that the mice uses to make a response, binned at `10ms`. 
* `dat['pupil']`: pupil area  (noisy, because pupil is very small) + pupil horizontal and vertical position. 
* `dat['lfp']`: recording of the local field potential in each brain area from this experiment, binned at `10ms`.
* `dat['brain_area_lfp']`: brain area names for the LFP channels. 
* `dat['trough_to_peak']`: measures the width of the action potential waveform for each neuron. Widths `<=10` samples are "putative fast spiking neurons". 
* `dat['waveform_w']`: temporal components of spike waveforms. `w@u` reconstructs the time by channels action potential shape. 
* `dat['waveform_u]`: spatial components of spike waveforms.
* `dat['%X%_passive']`: same as above for `X` = {`spks`, `lfp`, `pupil`, `wheel`, `contrast_left`, `contrast_right`} but for  passive trials at the end of the recording when the mouse was no longer engaged and stopped making responses. 




# Data Pre-process & Model Fitting --- neuron as feature
Since data shape is different in different sessions, loop all dataset for each session

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV

np.random.seed(31415)

# groupings of brain regions
regions = ["vis ctx", "thal", "hipp", "other ctx", "midbrain", "basal ganglia", "cortical subplate", "other"]
brain_groups = [["VISa", "VISam", "VISl", "VISp", "VISpm", "VISrl"], # visual cortex
                ["CL", "LD", "LGd", "LH", "LP", "MD", "MG", "PO", "POL", "PT", "RT", "SPF", "TH", "VAL", "VPL", "VPM"], # thalamus
                ["CA", "CA1", "CA2", "CA3", "DG", "SUB", "POST"], # hippocampal
                ["ACA", "AUD", "COA", "DP", "ILA", "MOp", "MOs", "OLF", "ORB", "ORBm", "PIR", "PL", "SSp", "SSs", "RSP"," TT"], # non-visual cortex
                ["APN", "IC", "MB", "MRN", "NB", "PAG", "RN", "SCs", "SCm", "SCig", "SCsg", "ZI"], # midbrain
                ["ACB", "CP", "GPe", "LS", "LSc", "LSr", "MS", "OT", "SNr", "SI"], # basal ganglia 
                ["BLA", "BMA", "EP", "EPd", "MEA"] # cortical subplate
                ]

for i in range(len(alldat)):
    dat = alldat[i]
    #============================================ Data Pre-process ============================================
    print('***************************** SECTION', i, '*****************************')

    # for debug
    # if i!=11:
    #     continue

    # ************* Data for visual stimulus and response *************
    response = dat['response'] # right - nogo - left (-1, 0, 1) [n_trial,]
    response_move = np.where(response!=0, 1, 0)  # 1 as have response, 0 as not

    vis_right = dat['contrast_right'] # 0 - low - high [n_trial,]
    vis_left = dat['contrast_left'] # 0 - low - high [n_trial,]
    vis_diff = vis_left - vis_right

    stim_ipsi = np.where(vis_left!=0, 1, 0)  # ipsilateral stimulus [n_trial,]
    stim_contr = np.where(vis_right!=0, 1, 0)  # contralateral stilumus [n_trial,]


    # ************* Data for spike fragments related with stimulus *************
    spks = dat['spks'].transpose((2,1,0))  # [n_time, n_trial, n_neuron] (250, 340, 698)
    # print(spks.shape)
    n_time, n_trial, n_neuron = spks.shape[0], spks.shape[1], spks.shape[2]

    idx_spks_stim_start = 44  # idx of starting visual stimulus
    idx_spks_stim_end = 84
    spks_frag_stim = spks[idx_spks_stim_start:idx_spks_stim_end, ...]  # fragment of spikes related with stimulus
    print(spks_frag_stim.shape)


    # ************* Data for spike fragments related with response *************
    response_time = dat['response_time']  # [n_trial,1] (340,1)
    idx_response_time = np.round(response_time.squeeze()/0.01).astype(int)  # [n_trial,] (340,)


    # fragment of spikes related with response (5, 340, 698)
    num_pre = 5  # num of time bin pre the response to collect spks
    for idx_trial in range(n_trial):  # using loop, should have a better way...
        if idx_trial==0:
            # spks_frag_response = spks[idx_response_time[idx_trial]-num_pre:idx_response_time[idx_trial], idx_trial, :]
            # spks_frag_response = spks_frag_response[:,np.newaxis,:]
            if idx_response_time[idx_trial]>n_time:  # response time may bigger than time bin
                spks_frag_response = spks[-num_pre:, idx_trial, :]
                spks_frag_response = spks_frag_response[:,np.newaxis,:]
                # spks_frag_response = None
                # continue
            else:
                spks_frag_response = spks[idx_response_time[idx_trial]-num_pre:idx_response_time[idx_trial], idx_trial, :]
                spks_frag_response = spks_frag_response[:,np.newaxis,:]
        else:
            # spks_frag_new = spks[idx_response_time[idx_trial]-num_pre:idx_response_time[idx_trial], idx_trial, :]
            # spks_frag_new = spks_frag_new[:,np.newaxis,:]
            # spks_frag_response = np.concatenate((spks_frag_response, spks_frag_new), axis=1)
            if idx_response_time[idx_trial]>n_time:  # response time may bigger than time bin
                spks_frag_new = spks[-num_pre:, idx_trial, :]
                spks_frag_new = spks_frag_new[:,np.newaxis,:]
                # continue
            else:
                spks_frag_new = spks[idx_response_time[idx_trial]-num_pre:idx_response_time[idx_trial], idx_trial, :]
                spks_frag_new = spks_frag_new[:,np.newaxis,:]
            spks_frag_response = np.concatenate((spks_frag_response, spks_frag_new), axis=1)
    print(spks_frag_response.shape)


    # ************* Data for spks in different brain regions *************
    brain_area = dat['brain_area']
    list_brain_region = len(brain_area)*[len(regions)]

    for idx_neuron in range(len(brain_area)):
        for idx_brain_group in range(len(brain_groups)):
            if np.isin(brain_area[idx_neuron], brain_groups[idx_brain_group]):
                list_brain_region[idx_neuron] = idx_brain_group
                break
            else:
                list_brain_region[idx_neuron] = len(brain_groups)
    unique_list_brain_region = np.unique(list_brain_region)
    print(i, unique_list_brain_region)

    # loop through all brain groups
    for idx_brain_region in range(len(unique_list_brain_region)):
        print('=============== Region', unique_list_brain_region[idx_brain_region], regions[idx_brain_region], '===============')
        idx_neuron_region = np.argwhere(list_brain_region==unique_list_brain_region[idx_brain_region]).squeeze()
        spks_frag_stim_region = spks_frag_stim[...,idx_neuron_region]
        spks_frag_response_region = spks_frag_response[...,idx_neuron_region]
        # if idx_brain_region==0:
        #     print(len(idx_neuron_region))
        #     spks_frag_stim_region = spks_frag_stim[...,idx_neuron_region]
        #     spks_frag_response_region = spks_frag_response[...,idx_neuron_region]
        #     print(spks_frag_stim_region.shape)
        #     print(spks_frag_response_region.shape)
        #============================================ Model Fitting ============================================
        # ************* Logistic reg for spks_frag_stim---stim_ipsi (L) *************
        # print(spks_frag_stim_region.shape)
        time_stim = spks_frag_stim_region.shape[0]
        list_score_stimL_train = []
        list_score_stimL_test = []
        for idx_time in range(time_stim):
            X_stimL = spks_frag_stim_region[idx_time]  # [n_trial, m_ueuron]
            y_stimL = stim_ipsi  # [n_trial,]

            X_stimL_train, X_stimL_test, y_stimL_train, y_stimL_test = train_test_split(X_stimL, y_stimL, test_size=0.2)
            log_reg_l1L = LogisticRegression(penalty="l1", C=1, solver="saga", max_iter=5000).fit(X_stimL_train, y_stimL_train)

            score_trainL = log_reg_l1L.score(X_stimL_train, y_stimL_train)
            score_testL = log_reg_l1L.score(X_stimL_test, y_stimL_test)
            list_score_stimL_train.append(score_trainL)
            list_score_stimL_test.append(score_testL)
        print('spks_frag_stim---stim_ipsi (L)')
        print(np.sum(list_score_stimL_train)/len(list_score_stimL_train))
        print(np.sum(list_score_stimL_test)/len(list_score_stimL_test))
        print('--------------------------------')


        # ************* Logistic reg for spks_frag_stim---stim_contr (R) *************
        time_stim = spks_frag_stim_region.shape[0]
        list_score_stimR_train = []
        list_score_stimR_test = []
        for idx_time in range(time_stim):
            X_stimR = spks_frag_stim_region[idx_time]  # [n_trial, m_ueuron]
            y_stimR = stim_contr  # [n_trial,]

            X_stimR_train, X_stimR_test, y_stimR_train, y_stimR_test = train_test_split(X_stimR, y_stimR, test_size=0.2)
            log_reg_l1R = LogisticRegression(penalty="l1", C=1, solver="saga", max_iter=5000).fit(X_stimR_train, y_stimR_train)

            score_trainR = log_reg_l1R.score(X_stimR_train, y_stimR_train)
            score_testR = log_reg_l1R.score(X_stimR_test, y_stimR_test)
            list_score_stimR_train.append(score_trainR)
            list_score_stimR_test.append(score_testR)

        print('spks_frag_stim---stim_contr (R)')
        print(np.sum(list_score_stimR_train)/len(list_score_stimR_train))
        print(np.sum(list_score_stimR_test)/len(list_score_stimR_test))
        print('--------------------------------')


        # ************* Logistic reg for spks_frag_response---response_move *************
        time_response = spks_frag_response_region.shape[0]
        list_score_move_train = []
        list_score_move_test = []
        for idx_time in range(time_response):
            X_move = spks_frag_response_region[idx_time]  # [n_trial, m_ueuron]
            y_move = response_move  # [n_trial,] (340,)

            X_move_train, X_move_test, y_move_train, y_move_test = train_test_split(X_move, y_move, test_size=0.2)
            log_reg_l1move = LogisticRegression(penalty="l1", C=1, solver="saga", max_iter=5000).fit(X_move_train, y_move_train)

            score_train_move = log_reg_l1move.score(X_move_train, y_move_train)
            score_test_move = log_reg_l1move.score(X_move_test, y_move_test)
            list_score_move_train.append(score_train_move)
            list_score_move_test.append(score_test_move)

        print('spks_frag_response---response_move')
        print(np.sum(list_score_move_train)/len(list_score_move_train))
        print(np.sum(list_score_move_test)/len(list_score_move_test))
        print('--------------------------------')


        # ************* Logistic reg for spks_frag_response---response *************
        # data prepare
        idx_response_LR = np.argwhere(response!=0).squeeze()
        response_LR = response[idx_response_LR]  # only take effective response trials [m_trial,]
        spks_frag_response_region_LR = spks_frag_response_region[:,idx_response_LR,:]  # [m_time, m_trials, m_neuron]

        time_response = spks_frag_response_region_LR.shape[0]
        list_score_responseLR_train = []
        list_score_responseLR_test = []
        for idx_time in range(time_response):
            X_responseLR = spks_frag_response_region_LR[idx_time]  # [m_trial, m_ueuron]
            y_responseLR = response_LR  # [m_trial,]

            X_responseLR_train, X_responseLR_test, y_responseLR_train, y_responseLR_test = train_test_split(X_responseLR, y_responseLR, test_size=0.2)
            log_reg_l1responseLR = LogisticRegression(penalty="l1", C=1, solver="saga", max_iter=5000).fit(X_responseLR_train, y_responseLR_train)

            score_train_responseLR = log_reg_l1responseLR.score(X_responseLR_train, y_responseLR_train)
            score_test_responseLR = log_reg_l1responseLR.score(X_responseLR_test, y_responseLR_test)
            list_score_responseLR_train.append(score_train_responseLR)
            list_score_responseLR_test.append(score_test_responseLR)

        print('spks_frag_response---response')
        print(np.sum(list_score_responseLR_train)/len(list_score_responseLR_train))
        print(np.sum(list_score_responseLR_test)/len(list_score_responseLR_test))
        print('--------------------------------')

***************************** SECTION 0 *****************************
(40, 214, 734)
(5, 214, 734)
0 [0 2 3 5 7]
=============== Region 0 vis ctx ===============
spks_frag_stim---stim_ipsi (L)
0.7634502923976607
0.5168604651162791
--------------------------------
spks_frag_stim---stim_contr (R)
0.7644736842105264
0.5430232558139535
--------------------------------
spks_frag_response---response_move
0.7859649122807018
0.5953488372093024
--------------------------------
spks_frag_response---response
0.8107142857142857
0.5428571428571428
--------------------------------
=============== Region 2 thal ===============
spks_frag_stim---stim_ipsi (L)
0.8232456140350877
0.5284883720930234
--------------------------------
spks_frag_stim---stim_contr (R)
0.8301169590643275
0.5319767441860466
--------------------------------
spks_frag_response---response_move
0.8608187134502924
0.6139534883720931
--------------------------------
spks_frag_response---response
0.8535714285714286
0.45
---------------

In [ ]:
n_neuron

734

In [ ]:
idx_neuron_region

array([363, 364, 365, 368, 369, 371, 374, 375, 376, 377, 379, 382, 383,
       385, 386, 389, 391, 392, 393, 396, 397, 399, 400, 401, 402, 403,
       406, 407, 409, 411, 412, 413, 415, 417, 418, 419, 420, 422, 423,
       425, 426, 428, 429, 431, 432, 434, 437, 438, 443, 444, 445, 447,
       449, 450, 453, 456, 461, 463, 465, 468, 470, 473, 474, 481, 482,
       483, 485, 486, 488, 489, 490, 491, 492, 495, 497, 498, 500, 501,
       504, 506, 507, 511, 513, 514, 516, 517, 520, 522, 524, 526, 528,
       529, 531, 532, 534, 535, 536, 538, 543, 544, 545, 546, 548, 549,
       551, 552, 554, 557, 561, 565, 566, 567, 570, 574, 575, 577, 583,
       584, 585, 586, 587, 588, 590, 600, 604, 605, 606, 608, 610, 611,
       613, 616, 618, 619, 622, 625, 628, 629, 630, 631, 633, 636, 637,
       639, 641, 642, 649, 650, 651, 652, 656, 662, 669, 670, 673, 675,
       681, 685, 687, 688, 689, 693, 694, 696, 697, 698, 699, 700, 703,
       705, 706, 712, 716, 717, 720, 721, 725, 732])

In [ ]:
spks_frag_response.shape

(250, 214, 5)

In [ ]:
response_time.max()

2.689699163224759

# Data Pre-process & Model Fitting v2.0 --- time as feature
Since data shape is different in different sessions, loop all dataset for each session

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV

np.random.seed(31415)

# groupings of brain regions
regions = ["vis ctx", "thal", "hipp", "other ctx", "midbrain", "basal ganglia", "cortical subplate", "other"]
brain_groups = [["VISa", "VISam", "VISl", "VISp", "VISpm", "VISrl"], # visual cortex
                ["CL", "LD", "LGd", "LH", "LP", "MD", "MG", "PO", "POL", "PT", "RT", "SPF", "TH", "VAL", "VPL", "VPM"], # thalamus
                ["CA", "CA1", "CA2", "CA3", "DG", "SUB", "POST"], # hippocampal
                ["ACA", "AUD", "COA", "DP", "ILA", "MOp", "MOs", "OLF", "ORB", "ORBm", "PIR", "PL", "SSp", "SSs", "RSP"," TT"], # non-visual cortex
                ["APN", "IC", "MB", "MRN", "NB", "PAG", "RN", "SCs", "SCm", "SCig", "SCsg", "ZI"], # midbrain
                ["ACB", "CP", "GPe", "LS", "LSc", "LSr", "MS", "OT", "SNr", "SI"], # basal ganglia 
                ["BLA", "BMA", "EP", "EPd", "MEA"] # cortical subplate
                ]

for i in range(len(alldat)):
    dat = alldat[i]
    #============================================ Data Pre-process ============================================
    print('*********** SECTION', i, '***********')

    # for debug
    # if i!=11:
    # if i<9 or i>12:
    #     continue

    # ************* Data for visual stimulus and response *************
    response = dat['response'] # right - nogo - left (-1, 0, 1) [340,] [n_trial,]
    response_move = np.where(response!=0, 1, 0)  # 1 as have response, 0 as not

    vis_right = dat['contrast_right'] # 0 - low - high [340,] [n_trial,]
    vis_left = dat['contrast_left'] # 0 - low - high [340,] [n_trial,]
    vis_diff = vis_left - vis_right

    stim_ipsi = np.where(vis_left!=0, 1, 0)  # ipsilateral stimulus [340,] [n_trial,]
    stim_contr = np.where(vis_right!=0, 1, 0)  # contralateral stilumus [340,] [n_trial,]


    # ************* Data for spike fragments related with stimulus *************
    spks = dat['spks']  # [n_neuron, n_trial, n_time] (698, 340, 250)
    n_neuron, n_trial, n_time = spks.shape[0], spks.shape[1], spks.shape[2]

    idx_spks_stim_start = 44  # idx of starting visual stimulus
    idx_spks_stim_end = 84
    spks_frag_stim = spks[..., idx_spks_stim_start:idx_spks_stim_end]  # fragment of spikes related with stimulus (698, 340, 20)


    # ************* Data for spike fragments related with response *************
    response_time = dat['response_time']  # [n_trial,1] (340,1)
    idx_response_time = np.round(response_time.squeeze()/0.01).astype(int)  # [n_trial,] (340,)


    # fragment of spikes related with response (5, 340, 698)
    response_time = dat['response_time']  # [n_trial,1] (340,1)
    idx_response_time = np.round(response_time.squeeze()/0.01).astype(int)  # [n_trial,] (340,)


    # fragment of spikes related with response (698, 340, 5)
    num_pre = 5  # num of time bin pre the response to collect spks
    # spks_frag_response = spks[..., idx_response_time]
    for idx_trial in range(n_trial):  # using loop, should have a better way...
        if idx_trial==0:
            if idx_response_time[idx_trial]>n_time:  # response time may bigger than time bin
                spks_frag_response = spks[:, idx_trial, -num_pre:]
                spks_frag_response = spks_frag_response[:,np.newaxis,:]
                # spks_frag_response = None
                # continue
            else:
                spks_frag_response = spks[:, idx_trial, idx_response_time[idx_trial]-num_pre:idx_response_time[idx_trial]]
                spks_frag_response = spks_frag_response[:,np.newaxis,:]
        else:
            if idx_response_time[idx_trial]>n_time:  # response time may bigger than time bin
                spks_frag_new = spks[:, idx_trial, -num_pre:]
                spks_frag_new = spks_frag_new[:,np.newaxis,:]
                # continue
            else:
                spks_frag_new = spks[:, idx_trial, idx_response_time[idx_trial]-num_pre:idx_response_time[idx_trial]]
                spks_frag_new = spks_frag_new[:,np.newaxis,:]
            spks_frag_response = np.concatenate((spks_frag_response, spks_frag_new), axis=1)
    # print(spks_frag_response.shape)

    # ************* Data for brain area *************
    brain_area = dat['brain_area']  # [n_neuron,]
    # print(brain_area.shape)



    #============================================ Model Fitting ============================================
    num_dominant_neuron = 10  # number of most important neurons to track
    # ************* Logistic reg for spks_frag_stim---stim_ipsi (L) *************
    list_score_stimL_train = []
    list_score_stimL_test = []
    for idx_neuron in range(n_neuron):
        X_stimL = spks_frag_stim[idx_neuron]  # [n_trial, n_time] (340, 250)
        y_stimL = stim_ipsi  # [n_trial,] (340,)

        X_stimL_train, X_stimL_test, y_stimL_train, y_stimL_test = train_test_split(X_stimL, y_stimL, test_size=0.2)
        log_reg_l1L = LogisticRegression(penalty="l1", C=1, solver="saga", max_iter=5000).fit(X_stimL_train, y_stimL_train)

        score_trainL = log_reg_l1L.score(X_stimL_train, y_stimL_train)
        score_testL = log_reg_l1L.score(X_stimL_test, y_stimL_test)
        list_score_stimL_train.append(score_trainL)
        list_score_stimL_test.append(score_testL)

    # print(max(list_score_stimL_train))
    # print(max(list_score_stimL_test))
    # print('-----------------------------')
    # print(min(list_score_stimL_train))
    # print(min(list_score_stimL_test))
    # print('-----------------------------')
    # print(sum(list_score_stimL_train)/n_neuron)
    # print(sum(list_score_stimL_test)/n_neuron)
    # print('-----------------------------')

    # sort corresponding brain area for high score neurons
    sortlist_score_stimL = np.sort(list_score_stimL_test)
    sortlist_neuron_idx_stimL = np.argsort(list_score_stimL_test)

    list_brain_area_L = brain_area[sortlist_neuron_idx_stimL[-1:-num_dominant_neuron:-1]]
    list_brain_group_L = len(list_brain_area_L)*[7]
    # print(list_brain_area_L)
    # print(list_brain_group_L)

    for idx_neuron in range(len(list_brain_area_L)):
        for idx_brain_group in range(len(brain_groups)):
            if np.isin(list_brain_area_L[idx_neuron], brain_groups[idx_brain_group]):
                list_brain_group_L[idx_neuron] = idx_brain_group
                break
            else:
                list_brain_group_L[idx_neuron] = len(brain_groups)
    print('spks_frag_stim---stim_ipsi (L)')
    print(list_brain_area_L)
    print(list_brain_group_L)
    print('------------------------------------')


    # ************* Logistic reg for spks_frag_stim---stim_contr (R) *************
    list_score_stimR_train = []
    list_score_stimR_test = []
    for idx_neuron in range(n_neuron):
        X_stimR = spks_frag_stim[idx_neuron]  # [n_trial, n_time] (340, 250)
        y_stimR = stim_contr  # [n_trial,] (340,)

        X_stimR_train, X_stimR_test, y_stimR_train, y_stimR_test = train_test_split(X_stimR, y_stimR, test_size=0.2)
        log_reg_l1R = LogisticRegression(penalty="l1", C=1, solver="saga", max_iter=5000).fit(X_stimR_train, y_stimR_train)

        score_trainR = log_reg_l1R.score(X_stimR_train, y_stimR_train)
        score_testR = log_reg_l1R.score(X_stimR_test, y_stimR_test)
        list_score_stimR_train.append(score_trainR)
        list_score_stimR_test.append(score_testR)

    # sort corresponding brain area for high score neurons
    sortlist_score_stimR = np.sort(list_score_stimR_test)
    sortlist_neuron_idx_stimR = np.argsort(list_score_stimR_test)

    list_brain_area_R = brain_area[sortlist_neuron_idx_stimR[-1:-num_dominant_neuron:-1]]
    list_brain_group_R = len(list_brain_area_R)*[7]
    # print(list_brain_area_R)
    # print(list_brain_group_R)

    for idx_neuron in range(len(list_brain_area_R)):
        for idx_brain_group in range(len(brain_groups)):
            if np.isin(list_brain_area_R[idx_neuron], brain_groups[idx_brain_group]):
                list_brain_group_R[idx_neuron] = idx_brain_group
                break
            else:
                list_brain_group_R[idx_neuron] = len(brain_groups)
    print('spks_frag_stim---stim_ipsi (R)')
    print(list_brain_area_R)
    print(list_brain_group_R)
    print('------------------------------------')


    # ************* Logistic reg for spks_frag_response---response_move *************
    list_score_move_train = []
    list_score_move_test = []
    for idx_neuron in range(n_neuron):
        X_move = spks_frag_response[idx_neuron]  # [n_trial, n_time] (340, 250)
        y_move = response_move  # [n_trial,] (340,)

        X_move_train, X_move_test, y_move_train, y_move_test = train_test_split(X_move, y_move, test_size=0.2)
        log_reg_l1move = LogisticRegression(penalty="l1", C=1, solver="saga", max_iter=5000).fit(X_move_train, y_move_train)

        score_train_move = log_reg_l1move.score(X_move_train, y_move_train)
        score_test_move = log_reg_l1move.score(X_move_test, y_move_test)
        list_score_move_train.append(score_train_move)
        list_score_move_test.append(score_test_move)

    # sort corresponding brain area for high score neurons
    sortlist_score_move = np.sort(list_score_move_test)
    sortlist_neuron_idx_move = np.argsort(list_score_move_test)

    list_brain_area_move = brain_area[sortlist_neuron_idx_move[-1:-num_dominant_neuron:-1]]
    list_brain_group_move = len(list_brain_area_move)*[7]
    # print(list_brain_area_move)
    # print(list_brain_group_move)

    for idx_neuron in range(len(list_brain_area_move)):
        for idx_brain_group in range(len(brain_groups)):
            if np.isin(list_brain_area_move[idx_neuron], brain_groups[idx_brain_group]):
                list_brain_group_move[idx_neuron] = idx_brain_group
                break
            else:
                list_brain_group_move[idx_neuron] = len(brain_groups)
    print('spks_frag_response---response_move')
    print(list_brain_area_move)
    print(list_brain_group_move)
    print('------------------------------------')


    # ************* Logistic reg for spks_frag_response---response *************
    # data prepare
    idx_response_LR = np.argwhere(response!=0).squeeze()
    response_LR = response[idx_response_LR]  # only take effective response trials [m_trial,] (276,)
    spks_frag_response_LR = spks_frag_response[:,idx_response_LR,:]  # [n_neuron, m_trials, n_time] (698, 276, 5)

    list_score_responseLR_train = []
    list_score_responseLR_test = []
    for idx_neuron in range(n_neuron):
        X_responseLR = spks_frag_response_LR[idx_neuron]  # [m_trial, n_time] (276, 250)
        y_responseLR = response_LR  # [m_trial,] (276,)

        X_responseLR_train, X_responseLR_test, y_responseLR_train, y_responseLR_test = train_test_split(X_responseLR, y_responseLR, test_size=0.2)
        log_reg_l1responseLR = LogisticRegression(penalty="l1", C=1, solver="saga", max_iter=5000).fit(X_responseLR_train, y_responseLR_train)

        score_train_responseLR = log_reg_l1responseLR.score(X_responseLR_train, y_responseLR_train)
        score_test_responseLR = log_reg_l1responseLR.score(X_responseLR_test, y_responseLR_test)
        list_score_responseLR_train.append(score_train_responseLR)
        list_score_responseLR_test.append(score_test_responseLR)

    # sort corresponding brain area for high score neurons
    sortlist_score_responseLR = np.sort(list_score_responseLR_test)
    sortlist_neuron_idx_responseLR = np.argsort(list_score_responseLR_test)

    list_brain_area_responseLR = brain_area[sortlist_neuron_idx_responseLR[-1:-num_dominant_neuron:-1]]
    list_brain_group_responseLR = len(list_brain_area_R)*[7]
    # print(list_brain_area_responseLR)
    # print(list_brain_group_responseLR)

    for idx_neuron in range(len(list_brain_area_responseLR)):
        for idx_brain_group in range(len(brain_groups)):
            if np.isin(list_brain_area_responseLR[idx_neuron], brain_groups[idx_brain_group]):
                list_brain_group_responseLR[idx_neuron] = idx_brain_group
                break
            else:
                list_brain_group_responseLR[idx_neuron] = len(brain_groups)
    print('spks_frag_response---response')
    print(list_brain_area_responseLR)
    print(list_brain_group_responseLR)
    print('------------------------------------')

    print('======================================================================')

*********** SECTION 0 ***********
spks_frag_stim---stim_ipsi (L)
['MOs' 'LS' 'DG' 'ACA' 'ACA' 'VISp' 'ACA' 'MOs' 'MOs']
[3, 5, 2, 3, 3, 0, 3, 3, 3]
------------------------------------
spks_frag_stim---stim_ipsi (R)
['MOs' 'MOs' 'MOs' 'VISp' 'root' 'MOs' 'ACA' 'CA3' 'VISp']
[3, 3, 3, 0, 7, 3, 3, 2, 0]
------------------------------------
spks_frag_response---response_move
['CA3' 'LS' 'VISp' 'CA3' 'LS' 'MOs' 'VISp' 'MOs' 'SUB']
[2, 5, 0, 2, 5, 3, 0, 3, 2]
------------------------------------
spks_frag_response---response
['MOs' 'MOs' 'LS' 'SUB' 'VISp' 'MOs' 'ACA' 'DG' 'VISp']
[3, 3, 5, 2, 0, 3, 3, 2, 0]
------------------------------------
*********** SECTION 1 ***********
spks_frag_stim---stim_ipsi (L)
['POST' 'root' 'VISl' 'root' 'VISl' 'VISl' 'CA1' 'VISpm' 'VISpm']
[2, 7, 0, 7, 0, 0, 2, 0, 0]
------------------------------------
spks_frag_stim---stim_ipsi (R)
['VISpm' 'VISpm' 'VISpm' 'VISl' 'VISl' 'VISl' 'VISl' 'VISl' 'VISl']
[0, 0, 0, 0, 0, 0, 0, 0, 0]
------------------------------

In [ ]:
idx_response_time[idx_trial]-num_pre

264

In [ ]:
response_time.max()

2.689699163224759

In [ ]:
# groupings of brain regions
regions = ["vis ctx", "thal", "hipp", "other ctx", "midbrain", "basal ganglia", "cortical subplate", "other"]
brain_groups = [["VISa", "VISam", "VISl", "VISp", "VISpm", "VISrl"], # visual cortex
                ["CL", "LD", "LGd", "LH", "LP", "MD", "MG", "PO", "POL", "PT", "RT", "SPF", "TH", "VAL", "VPL", "VPM"], # thalamus
                ["CA", "CA1", "CA2", "CA3", "DG", "SUB", "POST"], # hippocampal
                ["ACA", "AUD", "COA", "DP", "ILA", "MOp", "MOs", "OLF", "ORB", "ORBm", "PIR", "PL", "SSp", "SSs", "RSP"," TT"], # non-visual cortex
                ["APN", "IC", "MB", "MRN", "NB", "PAG", "RN", "SCs", "SCm", "SCig", "SCsg", "ZI"], # midbrain
                ["ACB", "CP", "GPe", "LS", "LSc", "LSr", "MS", "OT", "SNr", "SI"], # basal ganglia 
                ["BLA", "BMA", "EP", "EPd", "MEA"] # cortical subplate
                ]